Импортируем основные библиотеки и нвстраиваем вид отображения графиков Matplotlib

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scipy.stats import randint as randint
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold


ohe = OneHotEncoder(sparse = False)

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

Импортируем наши данные в df_train и df_test.

In [2]:
df_train = pd.read_csv('train.csv', sep=',', encoding='utf8', index_col='_id')
df_test = pd.read_csv('test.csv', sep=',', encoding='utf8', index_col='_id')

Проверяем, все ли загрузились данные.

In [ ]:
df_test.info()

In [ ]:
df_train.info()

## Описание признаков
`_id - идентификатор человека
age - возраст
job - тип работы ("admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
marital - семейное положение ("divorced" (разведенный и овдовевший),"married","single","unknown")
education - уровень образования ("basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
default - есть ли дефолт по кредиту ("no","yes","unknown")
housing - есть ли кредит на жилье ("no","yes","unknown")
loan - есть ли персональный кредит ("no","yes","unknown")
contact - контактный тип связи ("cellular","telephone")
month - последний контактный месяц года ("jan", "feb", "mar", ..., "nov", "dec")
day_of_week - последний контактный день недели ("mon","tue","wed","thu","fri")
duration - длительность последнего контакта в секундах. (Важное замечание: этот атрибут сильно влияет на таргет (например, если длительность - 0 сек., то таргет - 0). Тем не менее, продолжительность не известна до выполнения вызова.
campaign - количество контактов, выполненных для этого клиента в рамках текущей маркетинговой кампании (включая последний контакт)
pdays - количество дней прошедших последнего контакта с клиентом с предыдущей маркетинговой кампании (999 означает, что ранее с клиентом не контактировали)
previous - количество контактов, выполненных до текущей маркетинговой кампании для этого клиента
poutcome - результат предыдущей маркетинговой кампании ("failure","nonexistent","success")`
## Ниже приведены показатели окружающей среды и банка
`emp.var.rate - изменение уровня занятости (квартальный показатель)
cons.price.idx - индекс потребительских цен (ежемесячный показатель)
cons.conf.idx - индекс доверия потребителя (ежемесячный показатель)
euribor3m - курс Euribor (ежедневный показатель)
nr.employed - количество сотрудников (квартальный показатель)
target - целевая переменная (таргет) - согласился ли клиент на вклад (депозит)`

### Метрика для проверки ROC-AUC

In [3]:
def categorize_education(df_input):
    if df_input == 'unknown':
        return 0
    elif df_input == 'illiterate':
        return 1
    elif df_input == 'basic.4y':
        return 2
    elif df_input == 'basic.6y':
        return 3
    elif df_input == 'basic.9y':
        return 4
    elif df_input == 'high.school':
        return 5
    elif df_input == 'professional.course':
        return 6
    elif df_input == 'university.degree':
        return 7
def categorize_month(df_input):
    if df_input == 'jan':
        return 1
    elif df_input == 'feb':
        return 2
    elif df_input == 'mar':
        return 3
    elif df_input == 'apr':
        return 4
    elif df_input == 'may':
        return 5
    elif df_input == 'jun':
        return 6
    elif df_input == 'jul':
        return 7
    elif df_input == 'aug':
        return 8
    elif df_input == 'sep':
        return 9
    elif df_input == 'oct':
        return 10
    elif df_input == 'nov':
        return 11
    elif df_input == 'dec':
        return 12
def categorize_day_of_week(df_input):
    if df_input == 'mon':
        return 1
    elif df_input == 'tue':
        return 2
    elif df_input == 'wed':
        return 3
    elif df_input == 'thu':
        return 4
    elif df_input == 'fri':
        return 5

In [6]:
def preproc_data(df_input):
    df_output = df_input.copy()
   
    df_output['job'] = df_output['job'].map(df_output.groupby('job').size())
    df_output['marital'] = df_output['marital'].replace({'unknown': 0, 'single': 1, 'married': 2, 'divorced': 3})
    df_output['poutcome'] = df_output['poutcome'].replace({'failure': 0, 'success': 1, 'nonexistent': 2})
    df_output = df_output.drop(['marital'], axis = 1)
    df_output['default'] = df_output['default'].replace({'no': 0, 'yes': 1, 'unknown': 2})
    df_output['housing'] = df_output['housing'].replace({'no': 0, 'yes': 1, 'unknown': 2})
    df_output['loan'] = df_output['loan'].replace({'no': 0, 'yes': 1, 'unknown': 2})
    df_output['contact'] = df_output['contact'].replace({'cellular': 0, 'telephone': 1})
    
    df_output['education'] = df_output['education'].map(categorize_education)
    df_output['month'] = df_output['month'].map(categorize_month)
    df_output['day_of_week'] = df_output['day_of_week'].map(categorize_day_of_week)
    
    return df_output

In [7]:
df_train_preproc = df_train.pipe(preproc_data)
df_test_preproc = df_test.pipe(preproc_data)

In [8]:
#Проверяем, что правильно закодировали все категориальные признаки и отсутствие пропусков.
df_train_preproc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24712 entries, df7489733b004bbe40d3d37b34f82419 to 46d0d25dfd1def79632dd437261d0b5c
Data columns (total 20 columns):
age               24712 non-null int64
job               24712 non-null int64
education         24712 non-null int64
default           24712 non-null int64
housing           24712 non-null int64
loan              24712 non-null int64
contact           24712 non-null int64
month             24712 non-null int64
day_of_week       24712 non-null int64
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null int64
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64
dtypes: float64(5), int64(15)
memory usage: 4.0+

In [9]:
# Отделение целевой переменной.
X = df_train_preproc.iloc[:, :-1].values
y = df_train_preproc.iloc[:, -1].values

In [10]:
# При помощи функции train_test_split() разбиваем размеченную обучающую выборку на две — 
# 1) новая обучающая X_train и 2) контрольная (валидационная) X_valid.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

In [11]:
scaler = MinMaxScaler()
knn = KNeighborsClassifier(random_state=13)
dtc = DecisionTreeClassifier(random_state=13)
lr = LogisticRegression(random_state=13)
cv = StratifiedKFold(n_splits=5, random_state=13, shuffle=True)

In [3]:
list = randint(5,15)
print(list)

In [29]:
# Подбор гиперпараметров модели будем делать методом Random Search
# Начнём с KNN. 
#{'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}
scaler = MinMaxScaler()
knn = KNeighborsClassifier()
TfidfVectorizer
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2)),
    ('clf', knn),
])

knn_param_grid = {
    'n_neighbors': [5,6,7,8,9,10,15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']}

knn_model = GridSearchCV(knn_pipe, knn_param_grid,
                         n_jobs=-1, cv=cv, scoring='roc_auc')

knn_model.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=13, shuffle=True),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': [5, 6, 7, 8, 9, 10, 15], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [31]:
print(knn_model.best_params_)
print(knn_model.best_score_)

{'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}
0.9137136028590692


In [ ]:
!telegram-send 'success'

In [32]:
knn_roc_auc = roc_auc_score(y_test, knn_model.predict_proba(X_test)[:, 1])
print(knn_roc_auc)

0.9142025574960839


In [33]:
lr_pipe = Pipeline([
    ('scaler', scaler),
    ('model', lr)
])

lr_param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(0,3,50),
    'class_weight': [None, 'balanced']}

lr_model = GridSearchCV(lr_pipe, lr_param_grid,
                              n_jobs=-1, cv=cv, scoring='roc_auc')

KeyboardInterrupt: 

In [ ]:
lr_model.fit(X_train, y_train)

In [ ]:
print(lr_model.best_params_)
print(lr_model.best_score_)

In [ ]:
!telegram-send 'success'

In [ ]:
lr_roc_auc = roc_auc_score(y_test, lr_model.predict_proba(X_test)[:, 1])
print(lr_roc_auc)

In [ ]:
dtc_pipe = Pipeline([
    ('scaler', scaler),
    ('model', dtc)
])

dtc_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 10),
    'min_samples_split': randint(2, 100),
    'min_samples_leaf': randint(1, 100),
    'class_weight': [None, 'balanced'],
    'presort': [False, True]}

dtc_model = GridSearchCV(dtc_pipe, dtc_param_grid, n_jobs=-1,
                         cv=cv, scoring='roc_auc')

In [ ]:
dtc_model.fit(X_train, y_train)

print(dtc_model.best_params_)
print(dtc_model.best_score_)

In [ ]:
!telegram-send 'success'

In [ ]:
dtc_roc_auc = roc_auc_score(y_valid, dtc_model.predict_proba(X_valid)[:, 1])
print(dtc_roc_auc)

In [18]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)

scores_knn = cross_val_score(
    knn_model,
    X,
    y,
    scoring='roc_auc',
    cv=cv
)

In [19]:
!telegram-send 'success'

In [ ]:
scores_dtc = cross_val_score(
    dtc_model,
    X,
    y,
    scoring='roc_auc',
    cv=cv
)

In [ ]:
!telegram-send 'success'

In [ ]:
scores_lr = cross_val_score(
    lr_model,
    X,
    y,
    scoring='roc_auc',
    cv=cv
)

In [ ]:
!telegram-send 'success'

In [21]:
df_submission = pd.DataFrame()
df_submission.loc[:,'_id'] = df_test_preproc.reset_index()['_id']
df_submission.loc[:,'target'] = knn_model.predict_proba(df_test_preproc)[:, 1]
df_submission.to_csv('submission.csv', sep=',', index=False, encoding='utf-8')